In [12]:
import tensorflow as tf

In [92]:
import collections


import numpy as np
import random


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed,LSTM, Activation, RepeatVector, Bidirectional,Embedding,LayerNormalization, Lambda, dot, Activation, concatenate 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras import layers
 
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow.keras.backend as K


## Data

In [3]:
import pandas as pd

In [8]:
data=pd.read_csv("10Category10Articles.csv")

In [9]:
data.head()

,Unnamed: 0,category,headline,authors,link,short_description,date
0,0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,32,CRIME,Rachel Dolezal Faces Felony Charges For Welfar...,Carla Herreria,https://www.huffingtonpost.com/entry/rachel-do...,"State prosecutors say almost $84,000 had been ...",2018-05-25
2,40,CRIME,"Man Faces Charges After Pulling Knife, Stun Gu...",Jenna Amatulli,https://www.huffingtonpost.com/entry/man-knife...,"“We thought we were going to die,"" one of the ...",2018-05-25
3,42,CRIME,2 People Injured In Indiana School Shooting,Marina Fang,https://www.huffingtonpost.com/entry/indiana-m...,"A male student, believed to be the suspect, ha...",2018-05-25
4,185,CRIME,Maryland Police Charge 3 Church Leaders With P...,Carol Kuruvilla,https://www.huffingtonpost.com/entry/maryland-...,"The victims, who are now adults, were enrolled...",2018-05-23


In [10]:
titles = data['headline'].to_list()

In [11]:
titles[0]

'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV'

In [12]:
len(titles)

100

In [13]:
random.shuffle(titles)

In [14]:
titles[0]

"'Star Wars' Burlesque Show Storms West Coast With Sexy Stormtroopers"

In [15]:
titles=titles[:20000]

In [58]:
#hyperParams
num_words = 10000
maxlen = 100
embed_dim = 128
batch_size = 16

In [59]:
import collections

In [60]:
titles_counter = collections.Counter([word for sentence in titles for word in sentence.split()])

print('{} Words.'.format(len([word for sentence in titles for word in sentence.split()])))
print('{} unique words.'.format(len(titles_counter)))
print('10 Most common words in the titles:')
print('"' + '" "'.join(list(zip(*titles_counter.most_common(20)))[0]) + '"')

1104 Words.
750 unique words.
10 Most common words in the titles:
"To" "The" "In" "Of" "Trump" "On" "After" "This" "A" "Is" "For" "And" "At" "North" "New" "Women" "Korea" "You" "With" "Summit"


In [61]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

In [62]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        # Find the length of the longest sequence/sentence
        length = max([len(seq) for seq in x])
    
    return pad_sequences(sequences=x, maxlen=length, padding='post')

In [84]:
def preprocess(x):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_x = pad_sequences(preprocess_x, maxlen)

    return preprocess_x, x_tk

preproc_titles, titles_tokenizer = preprocess(titles)
    
max_title_sequence_length = preproc_titles.shape[1]

titles_vocab_size = len(titles_tokenizer.word_index)


print('Data Preprocessed')
print("Max Title length:", max_title_sequence_length)
print("Title vocabulary size:", titles_vocab_size)


Data Preprocessed
Max Title length: 100
Title vocabulary size: 745


In [64]:
# preproc_titles[0].shape

In [65]:
# np.expand_dims(preproc_titles, -1)

In [93]:
tokenizer = Tokenizer(num_words = num_words, split=' ')
tokenizer.fit_on_texts(data['headline'])
seqs = tokenizer.texts_to_sequences(data['headline'])
pad_seqs = pad_sequences(seqs, maxlen)

ATTENTION

In [94]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

## Encoder

In [98]:
encoder_inputs = Input(shape=(maxlen,), name='Encoder-Input')
emb_layer = Embedding(num_words, embed_dim,input_length = maxlen, name='Body-Word-Embedding', mask_zero=False)
x = emb_layer(encoder_inputs)

state_h = Bidirectional(LSTM(128, activation='relu', name='Encoder-Last-LSTM', return_sequences=True))(x)
norm = LayerNormalization(axis=1)(state_h)
state_h = Bidirectional(LSTM(128, activation='relu', name='Encoder-Last-LSTM',return_sequences=True))(norm)
norm = LayerNormalization(axis=1)(state_h)
state_h = Bidirectional(LSTM(128, activation='relu', name='Encoder-Last-LSTM',return_sequences=False))(norm)
norm = LayerNormalization(axis=1)(state_h)


att_out=attention()(norm)
encoder_model = Model(inputs=encoder_inputs, outputs=norm, name='Encoder-Model')
seq2seq_encoder_out = encoder_model(encoder_inputs)

In [99]:
decoded = RepeatVector(maxlen)(seq2seq_encoder_out)
decoder_lstm = Bidirectional(LSTM(128, return_sequences=True, name='Decoder-LSTM-before'),merge_mode='sum')
decoder_lstm_output = decoder_lstm(decoded)
norm = layers.LayerNormalization(axis=1)(decoder_lstm_output)
decoder_dense = Dense(num_words, activation='softmax', name='Final-Output-Dense-before')
decoder_outputs = decoder_dense(norm)


In [100]:
tf.debugging.set_log_device_placement(True)
seq2seq_Model = Model(encoder_inputs, decoder_outputs)
print(seq2seq_Model.summary())
seq2seq_Model.compile(optimizer=tf.keras.optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')
history = seq2seq_Model.fit(preproc_titles, np.expand_dims(preproc_titles, -1),
          batch_size=batch_size,
          epochs=10)

Model: "functional_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder-Input (InputLayer)   [(None, 100)]             0         
_________________________________________________________________
Encoder-Model (Functional)   (None, 256)               2332560   
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 100, 256)          0         
_________________________________________________________________
bidirectional_62 (Bidirectio (None, 100, 128)          394240    
_________________________________________________________________
layer_normalization_58 (Laye (None, 100, 128)          200       
_________________________________________________________________
Final-Output-Dense-before (D (None, 100, 10000)        1290000   
Total params: 4,017,000
Trainable params: 4,017,000
Non-trainable params: 0
___________________________________________

In [ ]:
sentences = ["Corona is bitch"]
seq,seq_tokenizer = tokenize(sentences)
pad_seq=pad_sequences(seq, maxlen=53, padding='post')
sentence_vec = encoder_model.predict(pad_seq)[0]

In [ ]:
sentence_vec

In [ ]:
sentence_vec.shape

## Save the trained model

In [101]:
seq2seq_Model.save('saveModel/LSTM_EncoderDecoderTrained') 
encoder_model.save('saveModel/LSTM_EncoderTrained') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saveModel/LSTM_EncoderDecoderTrained\assets


FailedPreconditionError: saveModel/LSTM_EncoderTrained is not a directory

## Loading Models

In [ ]:
Model = tf.keras.models.load_model('saveModel/LSTM_EncoderDecoderTrained')
Model.summary()

In [ ]:
encoder = tf.keras.models.load_model('saveModel/LSTM_EncoderTrained')
encoder.summary()

## Using models for inference

In [ ]:
sentences = ["Corona is bitch"]
seq,seq_tokenizer = tokenize(sentences)
pad_seq=pad_sequences(seq, maxlen=53, padding='post')
sentence_vec = encoder.predict(pad_seq)[0]
sentence_vec

In [ ]:
Articles_100=pd.read_csv('10Category10Articles.csv')

In [ ]:
Titles_100 = Articles_100['headline'].to_list()

In [ ]:
len(Titles_100)

In [ ]:
Titles_100[0]

## Latents for the 10 Articles of 10 categories

In [ ]:
ArticleLatents=[]
for sentence in Titles_100:
    seq,seq_tokenizer = tokenize(sentence)
    pad_seq=pad_sequences(seq, maxlen=53, padding='post')
    sentence_vec = encoder.predict(pad_seq)[0]
    ArticleLatents.append(sentence_vec)

In [ ]:
len(ArticleLatents)

## Dictionary mapping between article to latent

In [ ]:
Article2Latent = {}

In [ ]:
def mapArticle2Latent(Articles, Latents):
    for i in range(len(Articles)):
        Article2Latent[Articles[i]]=Latents[i]
    

In [ ]:
mapArticle2Latent(Titles_100,ArticleLatents)

## K means

In [ ]:
LatentArray=np.stack( ArticleLatents, axis=0 )

In [ ]:
LatentArray.shape

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=10, init='random',
    n_init=10, max_iter=300, 
    tol=0.001, random_state=0
)
Clusters = km.fit_predict(LatentArray)

In [ ]:
km.cluster_centers_.shape

In [ ]:
ClusterIDs=km.labels_

In [ ]:
ClusterIDs

In [ ]:
km.labels_.shape

In [ ]:
Articles_100.head(5)

# Cluster to Article's Category Mapping!

## Focusing on Category, Article, Cluster ID given by Kmeans

In [ ]:
FocusDataframe=Articles_100[["category", "headline"]]

In [ ]:
FocusDataframe.head(5)

In [ ]:
FocusDataframe['Cluster ID']=ClusterIDs.tolist()

In [ ]:
FocusDataframe.head(5)

In [ ]:
CategoryVSCluster=FocusDataframe[["category","Cluster ID"]]

## Display Category vs Cluster for the 10 categories we chose!

In [ ]:
cat=['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES']

In [ ]:
CategoryVSCluster.loc[CategoryVSCluster['category'] == cat[0]].head(10)

### PseudoCode for Score calculations
    1. Find Max num of times occured cluster id for each of the ten articles chosen in each category
    2. Consider that that Max occured cluster id as the category code
    3. Accuracy = (Num of times occured/10)*100

In [ ]:
Clusters=[]
Counts=[]
groupDF=[]
for i in range(10):
    groupDF.append(CategoryVSCluster.loc[CategoryVSCluster['category'] == cat[i]])
    sorted=groupDF[i].groupby('Cluster ID')['category'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(1)
    Clusters.append(sorted.iloc[0]['Cluster ID'])
    Counts.append(sorted.iloc[0]['count'])

In [ ]:
Clusters

In [ ]:
Counts

In [ ]:
cat

In [ ]:
def mapCategory2Cluster(category, Cluster):
    Category2Cluster={}
    for i in range(len(category)):
        Category2Cluster[category[i]]=Cluster[i]
    return Category2Cluster

In [ ]:
Category2Cluster=mapCategory2Cluster(cat,Clusters)

In [ ]:
Category2Cluster

## Observing above, Number of groups the whole 100 articles grouped are 5 and they are 3,4,5,8,9:

In [ ]:
set(Clusters)

In [ ]:
len(set(Clusters))

# LOL LMAO